### Task 3

In [18]:
from sklearn.cluster import KMeans

In [19]:
toronto.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.785730,-79.158750
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.765690,-79.175256
3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440


## SEecting Downtown Toronto borough

In [20]:
downtown_toronto = toronto[toronto['Borough']=="Downtown Toronto"]
downtown_toronto.reset_index(drop=True,inplace=True)

In [21]:
adress = 'Downtown Toronto'
x = Nominatim(user_agent="kartik")
geo = x.geocode(adress)

down_lat = geo.latitude
down_lng = geo.longitude

In [22]:
down_map =  folium.Map([down_lat,down_lng], zoom_start = 12)

In [23]:
for lat , lng , neighborhood in zip(downtown_toronto['Latitude'],downtown_toronto['Longitude'],downtown_toronto['Neighborhood']):
    label = folium.Popup(neighborhood, parse_html=True)
    folium.CircleMarker([lat, lng],
                        radius=5,
                        popup=label,
                        color='blue',
                        fill=True,
                        fill_color='#3186cc',
                        fill_opacity=0.7,
                        parse_html=False).add_to(down_map)

In [24]:
CLIENT_ID = 'UUDATNQLSPQ3VLPACBR2Q2Q0BFDCM0ZXNM0D4T24R2TKN2WS' # your Foursquare ID
CLIENT_SECRET = 'ISR2D5ZTUDEZ5FHP3OEZNFB4HJCHUBFTDOVE5KHKL2OALHID' # your Foursquare Secret
VERSION = '20190421' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UUDATNQLSPQ3VLPACBR2Q2Q0BFDCM0ZXNM0D4T24R2TKN2WS
CLIENT_SECRET:ISR2D5ZTUDEZ5FHP3OEZNFB4HJCHUBFTDOVE5KHKL2OALHID


In [25]:
downtown_toronto.loc[0,'Neighborhood']

'Rosedale'

## lets get top 100 venue near Rosedale

In [26]:
neighborhood_lat = downtown_toronto.loc[0,'Latitude']
neighborhood_lng = downtown_toronto.loc[0,'Longitude']


In [27]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_lat, 
    neighborhood_lng, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=UUDATNQLSPQ3VLPACBR2Q2Q0BFDCM0ZXNM0D4T24R2TKN2WS&client_secret=ISR2D5ZTUDEZ5FHP3OEZNFB4HJCHUBFTDOVE5KHKL2OALHID&v=20190421&ll=43.681940000000054,-79.37847416699998&radius=500&limit=100'

In [28]:
results = requests.get(url).json()

In [29]:

venue =results['response']['groups'][0]['items']

In [30]:
from pandas.io.json import json_normalize

In [31]:
nearby_venue = json_normalize(venue)

In [32]:
nearby_venue['venue.categories'][0][0].keys()

dict_keys(['name', 'id', 'primary', 'icon', 'pluralName', 'shortName'])

In [33]:
def get_category(row):
    try:
        category_list = row['categories']
    except:
        category_list = row['venue.categories']
    if len(category_list) == 0:
        return None
    else:
        return category_list[0]['name']

In [34]:
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venue =nearby_venue.loc[:, filtered_columns]

In [35]:
nearby_venue

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,TD Canada Trust - Closed,"[{'name': 'Bank', 'id': '4bf58dd8d48988d10a951...",43.686094,-79.376549
1,Rosedale Park,"[{'name': 'Playground', 'id': '4bf58dd8d48988d...",43.682328,-79.378934
2,Rosedale Tennis Club,"[{'name': 'Tennis Court', 'id': '4e39a956bd410...",43.683226,-79.378984
3,Whitney Park,"[{'name': 'Park', 'id': '4bf58dd8d48988d163941...",43.682036,-79.373788
4,Mooredale House,"[{'name': 'Building', 'id': '4bf58dd8d48988d13...",43.678631,-79.380091


In [36]:
nearby_venue['venue.categories'] = nearby_venue.apply(get_category, axis=1)

In [37]:
nearby_venue

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,TD Canada Trust - Closed,Bank,43.686094,-79.376549
1,Rosedale Park,Playground,43.682328,-79.378934
2,Rosedale Tennis Club,Tennis Court,43.683226,-79.378984
3,Whitney Park,Park,43.682036,-79.373788
4,Mooredale House,Building,43.678631,-79.380091


In [38]:

downtown_toronto.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.681940,-79.378474
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.668160,-79.366602
2,M4Y,Downtown Toronto,Church and Wellesley,43.666585,-79.381302
3,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.655120,-79.362640
4,M5B,Downtown Toronto,"Ryerson,Garden District",43.657363,-79.378180


## Explore all neighborhood near Downtown

In [39]:
def get_nearby_venue(names, latitudes, longitudes, radius=500):
    venue_list = []
    for name, lat , lng in zip(names, latitudes, longitudes):
        print(name)
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venue_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']
        ) for v in results])
    
    nearby_venues = pd.DataFrame([cols for x in venue_list for  cols in x])
    nearby_venues.columns = ['Neighborhood' , 'Neighborhood Latitude',
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)
        

In [40]:
downtown_venues = get_nearby_venue(downtown_toronto['Neighborhood'] , downtown_toronto['Latitude'] , downtown_toronto['Longitude'])

Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie


In [41]:
downtown_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.68194,-79.378474,TD Canada Trust - Closed,43.686094,-79.376549,Bank
1,Rosedale,43.68194,-79.378474,Rosedale Park,43.682328,-79.378934,Playground
2,Rosedale,43.68194,-79.378474,Rosedale Tennis Club,43.683226,-79.378984,Tennis Court
3,Rosedale,43.68194,-79.378474,Whitney Park,43.682036,-79.373788,Park
4,Rosedale,43.68194,-79.378474,Mooredale House,43.678631,-79.380091,Building


## Analyze each neighborhood

In [42]:
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

In [43]:
downtown_onehot['Neighborhood'] = downtown_venues['Neighborhood']

In [44]:
fixed_col = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])

In [45]:
downtown_onehot = downtown_onehot[fixed_col]

In [46]:
downtown_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
downtown_onehot.shape

(1252, 185)

## Merge the details

In [48]:
downtown_grouped = downtown_onehot.groupby('Neighborhood').mean().reset_index()

In [49]:
downtown_grouped.head()

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide,King,Richmond",0.000000,0.0,0.0,0.030000,0.010000,0.00,0.0,0.030000,0.0,...,0.0,0.0,0.00,0.0,0.010000,0.000000,0.00,0.00,0.01,0.0
1,Berczy Park,0.000000,0.0,0.0,0.000000,0.016129,0.00,0.0,0.000000,0.0,...,0.0,0.0,0.00,0.0,0.016129,0.000000,0.00,0.00,0.00,0.0
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.014706,0.0,0.0,0.014706,0.000000,0.00,0.0,0.014706,0.0,...,0.0,0.0,0.00,0.0,0.000000,0.014706,0.00,0.00,0.00,0.0
3,"Cabbagetown,St. James Town",0.000000,0.0,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.0,...,0.0,0.0,0.00,0.0,0.000000,0.000000,0.00,0.00,0.00,0.0
4,Central Bay Street,0.000000,0.0,0.0,0.010000,0.010000,0.01,0.0,0.000000,0.0,...,0.0,0.0,0.01,0.0,0.010000,0.000000,0.01,0.01,0.01,0.0


In [50]:
z = downtown_grouped[downtown_grouped['Neighborhood'] == 'Central Bay Street'].T.reset_index()

In [51]:
z.columns = ['Venue', 'Frequency']

In [52]:
z = z.iloc[1:,]

In [53]:
z = z.sort_values('Frequency' , ascending=False).reset_index(drop=True)

In [54]:
def split(row):
    try:
        category_list = row['Venue']
    except:
        category_list = row['venue.categories']
    if len(category_list) == 0:
        return None
    else:
        return (category_list.split('_')[1])


In [55]:
z.head()

,Venue,Frequency
0,Coffee Shop,0.08
1,Clothing Store,0.07
2,Cosmetics Shop,0.04
3,Tea Room,0.03
4,Sushi Restaurant,0.03


In [56]:
downtown_grouped.head()

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide,King,Richmond",0.000000,0.0,0.0,0.030000,0.010000,0.00,0.0,0.030000,0.0,...,0.0,0.0,0.00,0.0,0.010000,0.000000,0.00,0.00,0.01,0.0
1,Berczy Park,0.000000,0.0,0.0,0.000000,0.016129,0.00,0.0,0.000000,0.0,...,0.0,0.0,0.00,0.0,0.016129,0.000000,0.00,0.00,0.00,0.0
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.014706,0.0,0.0,0.014706,0.000000,0.00,0.0,0.014706,0.0,...,0.0,0.0,0.00,0.0,0.000000,0.014706,0.00,0.00,0.00,0.0
3,"Cabbagetown,St. James Town",0.000000,0.0,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.0,...,0.0,0.0,0.00,0.0,0.000000,0.000000,0.00,0.00,0.00,0.0
4,Central Bay Street,0.000000,0.0,0.0,0.010000,0.010000,0.01,0.0,0.000000,0.0,...,0.0,0.0,0.01,0.0,0.010000,0.000000,0.01,0.01,0.01,0.0


## Return common venues from Neighborhood

In [57]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [58]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Hotel,Bar,Bakery,Restaurant,Burger Joint,Asian Restaurant,Gastropub,American Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Hotel,Farmers Market,Steakhouse,Café,Cheese Shop,Bakery,Breakfast Spot
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Coffee Shop,Italian Restaurant,Restaurant,Pub,Café,Bar,Speakeasy,Bakery,Park,Sandwich Place
3,"Cabbagetown,St. James Town",Coffee Shop,Park,Restaurant,Café,Pizza Place,Bakery,Italian Restaurant,Pub,Butcher,Liquor Store
4,Central Bay Street,Coffee Shop,Clothing Store,Cosmetics Shop,Middle Eastern Restaurant,Sushi Restaurant,Tea Room,Plaza,Fast Food Restaurant,Spa,Sporting Goods Shop


# Clustering

In [59]:
kclusters = 5

downtown_grouped_clustering = downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 4, 1, 1, 1], dtype=int32)

In [60]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = downtown_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_merged.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.681940,-79.378474,3,Playground,Park,Building,Bank,Tennis Court,Dance Studio,Deli / Bodega,Creperie,Falafel Restaurant,Exhibit
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.668160,-79.366602,1,Coffee Shop,Park,Restaurant,Café,Pizza Place,Bakery,Italian Restaurant,Pub,Butcher,Liquor Store
2,M4Y,Downtown Toronto,Church and Wellesley,43.666585,-79.381302,1,Coffee Shop,Japanese Restaurant,Gay Bar,Restaurant,Sushi Restaurant,Dance Studio,Grocery Store,Gastropub,Gym,Pub
3,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.655120,-79.362640,0,Coffee Shop,Restaurant,Breakfast Spot,Yoga Studio,Bakery,Health Food Store,Italian Restaurant,Food Truck,Event Space,Mexican Restaurant
4,M5B,Downtown Toronto,"Ryerson,Garden District",43.657363,-79.378180,1,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Bar,Italian Restaurant,Hotel,Diner,Plaza


In [61]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [62]:
map_clusters = folium.Map(location=[down_lat, down_lng], zoom_start=10)

## Maping clusters on Map

In [63]:

map_clusters = folium.Map(location=[down_lat, down_lng], zoom_start=10)


# add markers to the map

markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighborhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       

NameError: name 'rainbow' is not defined

In [ ]:
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [ ]:
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighborhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)



# Finish